# MindQuantum全面介绍

In [1]:
import os
os.environ['OMP_NUM_THREADS'] = '1'

In [2]:
from mindquantum import *
import numpy as np

## 量子门

In [3]:
H

H

In [4]:
H.on(0)

H(0)

In [5]:
H.matrix()

array([[ 0.70710678,  0.70710678],
       [ 0.70710678, -0.70710678]])

In [6]:
X.on(1,0)

X(1 <-: 0)

In [7]:
RX(np.pi)

RX(π)

In [8]:
RX(np.pi).matrix()

array([[6.123234e-17+0.j, 0.000000e+00-1.j],
       [0.000000e+00-1.j, 6.123234e-17+0.j]])

In [9]:
RX('a')

RX(a)

In [10]:
RX('a').matrix({'a':np.pi})

array([[6.123234e-17+0.j, 0.000000e+00-1.j],
       [0.000000e+00-1.j, 6.123234e-17+0.j]])

## 量子线路

In [11]:
circuit = Circuit()

In [12]:
circuit = Circuit([H.on(0), X.on(1, 0)])
circuit

q0: ──H────●──
           │  
q1: ───────X──

In [13]:
circuit += qft(range(3))
circuit

q0: ──H────●────H────PS(π/2)────PS(π/4)─────────────────────────@──
           │            │          │                            │  
q1: ───────X────────────●──────────┼───────H────PS(π/2)─────────┼──
                                   │               │            │  
q2: ───────────────────────────────●───────────────●───────H────@──

In [14]:
circuit += ZZ('alpha').on([0, 1])
circuit

q0: ──H────●────H────PS(π/2)────PS(π/4)─────────────────────────@────ZZ(alpha)──
           │            │          │                            │        │      
q1: ───────X────────────●──────────┼───────H────PS(π/2)─────────┼────ZZ(alpha)──
                                   │               │            │               
q2: ───────────────────────────────●───────────────●───────H────@───────────────

In [15]:
circuit += BARRIER
circuit += H.on(2)

In [16]:
circuit

q0: ──H────●────H────PS(π/2)────PS(π/4)─────────────────────────@────ZZ(alpha)───────
           │            │          │                            │        │           
q1: ───────X────────────●──────────┼───────H────PS(π/2)─────────┼────ZZ(alpha)───────
                                   │               │            │                    
q2: ───────────────────────────────●───────────────●───────H────@─────────────────H──

In [17]:
circuit += H.on(0)
circuit

q0: ──H────●────H────PS(π/2)────PS(π/4)─────────────────────────@────ZZ(alpha)────H──
           │            │          │                            │        │           
q1: ───────X────────────●──────────┼───────H────PS(π/2)─────────┼────ZZ(alpha)───────
                                   │               │            │                    
q2: ───────────────────────────────●───────────────●───────H────@─────────────────H──

In [18]:
circuit += BarrierGate(show=False)
circuit += Measure('q1').on(1)
circuit

q0: ──H────●────H────PS(π/2)────PS(π/4)─────────────────────────@────ZZ(alpha)────H───────────
           │            │          │                            │        │                    
q1: ───────X────────────●──────────┼───────H────PS(π/2)─────────┼────ZZ(alpha)─────────M(q1)──
                                   │               │            │                             
q2: ───────────────────────────────●───────────────●───────H────@─────────────────H───────────

## Simulator

In [19]:
sim = Simulator('projectq', 3)
sim

projectq simulator with 3 qubits.
Current quantum state:
1¦000⟩

In [20]:
print(sim.get_qs(True))

1¦000⟩


In [21]:
sim.apply_circuit(qft(range(3)))
sim

projectq simulator with 3 qubits.
Current quantum state:
√2/4¦000⟩
√2/4¦001⟩
√2/4¦010⟩
√2/4¦011⟩
√2/4¦100⟩
√2/4¦101⟩
√2/4¦110⟩
√2/4¦111⟩

In [22]:
sim.reset()
sim

projectq simulator with 3 qubits.
Current quantum state:
1¦000⟩

In [23]:
circ = Circuit().h(0).rx('a', 0).x(1,0).rx('b', 0)
circ.barrier(False).measure_all()
circ += Measure('q1_1').on(0)
circ

q0: ──H────RX(a)────●────RX(b)────M(q0)────M(q1_1)──
                    │                               
q1: ────────────────X─────────────M(q1)─────────────

In [24]:
sim = Simulator('projectq', circ.n_qubits)
res = sim.sampling(circ, {'a': np.pi, 'b': np.pi}, shots=100, seed=42)
res

shots: 100
Keys: q1_1 q1 q0│0.00    0.14        0.28        0.42        0.56         0.7
────────────────┼───────────┴───────────┴───────────┴───────────┴───────────┴
             010│▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒
                │                                                            
             101│▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
                │                                                            
{'010': 44, '101': 56}

In [25]:
sim.apply_circuit(circ[:4], {'a': np.pi, 'b': np.pi})
print(sim.get_qs(ket=True))

-√2/2¦01⟩
-√2/2¦10⟩


In [26]:
print(circ[:4].get_qs(ket=True, pr={'a':np.pi, 'b':np.pi}))

-√2/2¦01⟩
-√2/2¦10⟩


In [27]:
qft(range(3)).get_qs('projectq')

array([0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j,
       0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j])